<a href="https://colab.research.google.com/github/thenametochange/AlexNet/blob/main/AlextNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

导入package

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F

数据预处理

In [2]:
transform = transforms.Compose(
    [transforms.Resize((227, 227)),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
)

加载数据集

In [4]:
trainset = torchvision.datasets.CIFAR10(root='.data', train = True,
                      download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128,
                      shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                      download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=128,
                      shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified


100%|██████████| 170498071/170498071 [00:01<00:00, 99480014.81it/s] 


Extracting ./data/cifar-10-python.tar.gz to ./data


定义AlexNet模型

In [14]:
class AlexNet(nn.Module):
  def __init__(self, num_classes=10):
    super(AlexNet, self).__init__()
    self.features = nn.Sequential(
        nn.Conv2d(3, 96, kernel_size=11, stride=4, padding=2),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=3, stride=2),
        nn.Conv2d(96, 256, kernel_size=5, padding=2),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=3, stride=2),
        nn.Conv2d(256, 384, kernel_size=3, padding=1),
        nn.ReLU(inplace=True),
        nn.Conv2d(384, 384, kernel_size=3, padding=1),
        nn.ReLU(inplace=True),
        nn.Conv2d(384, 256, kernel_size=3, padding=1),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=3, stride=2),
    )
    self.classifier = nn.Sequential(
        nn.Dropout(),
        nn.Linear(256 * 6 * 6, 4096),
        nn.ReLU(inplace=True),
        nn.Dropout(),
        nn.Linear(4096, 4096),
        nn.ReLU(inplace=True),
        nn.Linear(4096, num_classes),
    )

  def forward(self, x):
    x = self.features(x)
    x = torch.flatten(x, 1)
    x = self.classifier(x)
    return x

初始化

In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
net = AlexNet().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9, weight_decay=0.0005)

训练模型

In [16]:
def train(epoch):
  net.train()
  running_loss = 0.0
  for i, data in enumerate(trainloader, 0):
    inputs, labels = data
    inputs, labels = inputs.to(device), labels.to(device)

    optimizer.zero_grad()
    outputs = net(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    running_loss += loss.item()
    if i % 100 == 99:
      print(f'[Epoch {epoch + 1}, Batch {i + 1}] loss: running_loss / 100:.3f')
      running_loss = 0.0

测试模型

In [17]:
def test():
  net.eval()
  correct = 0
  total = 0
  with torch.no_grad():
    for data in testloader:
      images, labels = data
      images, labels = images.to(device), labels.to(device)
      outputs = net(images)
      _, predicted = torch.max(outputs.data, 1)
      total += labels.size(0)
      correct += (predicted == labels).sum().item()

  print(f'Accuracy of the network on the 10000 test images: {100 * correct / total:.2f}%')


运行训练和测试

In [18]:
for epoch in range(10):
  train(epoch)
  test()

[Epoch 1, Batch 100] loss: running_loss / 100:.3f
[Epoch 1, Batch 200] loss: running_loss / 100:.3f
[Epoch 1, Batch 300] loss: running_loss / 100:.3f
Accuracy of the network on the 10000 test images: 34.99%
[Epoch 2, Batch 100] loss: running_loss / 100:.3f
[Epoch 2, Batch 200] loss: running_loss / 100:.3f
[Epoch 2, Batch 300] loss: running_loss / 100:.3f
Accuracy of the network on the 10000 test images: 50.74%
[Epoch 3, Batch 100] loss: running_loss / 100:.3f
[Epoch 3, Batch 200] loss: running_loss / 100:.3f
[Epoch 3, Batch 300] loss: running_loss / 100:.3f
Accuracy of the network on the 10000 test images: 62.47%
[Epoch 4, Batch 100] loss: running_loss / 100:.3f
[Epoch 4, Batch 200] loss: running_loss / 100:.3f
[Epoch 4, Batch 300] loss: running_loss / 100:.3f
Accuracy of the network on the 10000 test images: 68.26%
[Epoch 5, Batch 100] loss: running_loss / 100:.3f
[Epoch 5, Batch 200] loss: running_loss / 100:.3f
[Epoch 5, Batch 300] loss: running_loss / 100:.3f
Accuracy of the networ